# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
1/7/2024	Exploring Scaffolding Techniques for Agent-Administered Brief Cognitive Screening in Hospital Settings	DIS '24 Companion	"This study explores effective scaffolding strategies used by clinicians to assist in cognitive screening of hospitalized older patients, highlighting six key strategies identified through empirical research on the Abbreviated Mental Test (AMT) process, with implications for designing conversational agents in this context."	"Dingdong Liu, Sensen Gao, Zixin Chen, Yifan Shen, Chuhan Shi, Bertram E. Shi, and Xiaojuan Ma. 2024. Exploring Scaffolding Techniques for Agent-Administered Brief Cognitive Screening in Hospital Settings. In Companion Publication of the 2024 ACM Designing Interactive Systems Conference (DIS '24 Companion). Association for Computing Machinery, New York, NY, USA, 185-189. https://doi.org/10.1145/3656156.3663697"	DIS24-Scaffolding	https://dingdong-liu.github.io/files/DIS24-

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,1/7/2024,Exploring Scaffolding Techniques for Agent-Adm...,DIS '24 Companion,This study explores effective scaffolding stra...,"Dingdong Liu, Sensen Gao, Zixin Chen, Yifan Sh...",DIS24-Scaffolding,https://dingdong-liu.github.io/files/DIS24-Sca...,https://dingdong-liu.github.io/files/DIS24_Pos...
1,19/4/2024,CoArgue : Fostering Lurkers' Contribution to C...,CHI' 23,This research investigates obstacles hindering...,"Chengzhong Liu, Shixu Zhou, Dingdong Liu, Junz...",CHI23-CoArgue,https://dl.acm.org/doi/10.1145/3544548.3580932,NaN
2,13/6/2024,Exploring the Effects of Self-Mockery to Impro...,DIS' 22,This study explores the impact of self-mockery...,"Chengzhong Liu, Shixu Zhou, Yuanhao Zhang, Din...",DIS22-Self-Mockery,https://dl.acm.org/doi/10.1145/3532106.3533461,NaN
3,22/11/2022,PlanHelper: Supporting Activity Plan Construct...,CSCW '22,This study addresses challenges in using answe...,"Chengzhong Liu, Zeyu Huang, Dingdong Liu, Shix...",CSCW22-PlanHelper,https://dl.acm.org/doi/10.1145/3555555,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2022-CSCW22-PlanHelper.md  2024-DIS22-Self-Mockery.md
2024-CHI23-CoArgue.md      2024-DIS24-Scaffolding.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
